# Experiment 4: Recursive Best-First Search (RBFS)

In [1]:
import math

# Maze: 0 = free, 1 = wall
maze = [
    [0, 0, 0, 1, 0],
    [1, 1, 0, 1, 0],
    [0, 0, 0, 0, 0],
    [0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0]
]

start = (0, 0)
goal = (4, 4)
rows, cols = len(maze), len(maze[0])

expanded_nodes = 0

def heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def get_neighbors(node):
    x, y = node
    moves = [(1,0), (-1,0), (0,1), (0,-1)]
    result = []
    for dx, dy in moves:
        nx, ny = x + dx, y + dy
        if 0 <= nx < rows and 0 <= ny < cols and maze[nx][ny] == 0:
            result.append((nx, ny))
    return result

def rbfs(node, goal, path, g, f_limit):
    global expanded_nodes
    expanded_nodes += 1

    if node == goal:
        return path, g

    successors = []
    for s in get_neighbors(node):
        if s not in path:
            f = max(g + 1 + heuristic(s, goal), g + heuristic(node, goal))
            successors.append([s, f])

    if not successors:
        return None, math.inf

    while True:
        successors.sort(key=lambda x: x[1])
        best = successors[0]

        if best[1] > f_limit:
            return None, best[1]

        alternative = successors[1][1] if len(successors) > 1 else math.inf
        result, best[1] = rbfs(
            best[0], goal, path + [best[0]], g + 1, min(f_limit, alternative)
        )

        if result is not None:
            return result, best[1]

# Run RBFS
path, cost = rbfs(start, goal, [start], 0, math.inf)

print("RBFS Path:", path)
print("Path Cost:", cost)
print("Nodes Expanded:", expanded_nodes)


RBFS Path: [(0, 0), (0, 1), (0, 2), (1, 2), (2, 2), (2, 3), (2, 4), (3, 4), (4, 4)]
Path Cost: 8
Nodes Expanded: 9


# Experiment 5: Iterative Deepening A* (IDA*)

In [2]:
expanded_nodes = 0

def ida_star(start, goal):
    threshold = heuristic(start, goal)
    path = [start]

    while True:
        temp = search(path, 0, threshold)
        if temp == "FOUND":
            return path
        if temp == math.inf:
            return None
        threshold = temp

def search(path, g, threshold):
    global expanded_nodes
    node = path[-1]
    f = g + heuristic(node, goal)

    if f > threshold:
        return f
    if node == goal:
        return "FOUND"

    expanded_nodes += 1
    min_cost = math.inf

    for s in get_neighbors(node):
        if s not in path:
            path.append(s)
            temp = search(path, g + 1, threshold)
            if temp == "FOUND":
                return "FOUND"
            if temp < min_cost:
                min_cost = temp
            path.pop()

    return min_cost

path = ida_star(start, goal)

print("IDA* Path:", path)
print("Path Cost:", len(path) - 1)
print("Nodes Expanded:", expanded_nodes)


IDA* Path: [(0, 0), (0, 1), (0, 2), (1, 2), (2, 2), (2, 3), (2, 4), (3, 4), (4, 4)]
Path Cost: 8
Nodes Expanded: 8


# Experiment 6: Simplified Memory-Bounded A* (SMA*)

In [3]:
import heapq

MEMORY_LIMIT = 20
expanded_nodes = 0

class Node:
    def __init__(self, state, parent=None, g=0):
        self.state = state
        self.parent = parent
        self.g = g
        self.h = heuristic(state, goal)
        self.f = self.g + self.h

    def __lt__(self, other):
        return self.f < other.f

def reconstruct_path(node):
    path = []
    while node:
        path.append(node.state)
        node = node.parent
    return path[::-1]

def sma_star(start, goal):
    global expanded_nodes

    open_list = []
    root = Node(start)
    heapq.heappush(open_list, root)

    while open_list:
        if len(open_list) > MEMORY_LIMIT:
            open_list.sort(key=lambda x: x.f, reverse=True)
            open_list.pop()

        current = heapq.heappop(open_list)
        expanded_nodes += 1

        if current.state == goal:
            return reconstruct_path(current)

        for s in get_neighbors(current.state):
            child = Node(s, current, current.g + 1)
            heapq.heappush(open_list, child)

    return None

path = sma_star(start, goal)

print("SMA* Path:", path)
print("Path Cost:", len(path) - 1)
print("Nodes Expanded:", expanded_nodes)


SMA* Path: [(0, 0), (0, 1), (0, 2), (1, 2), (2, 2), (2, 3), (2, 4), (3, 4), (4, 4)]
Path Cost: 8
Nodes Expanded: 9
